<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_6_Tool_Using_Agent_Pattern_(LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  STEP 1: Imports

In [1]:
!pip install langchain langgraph langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 19.9 MB/s eta 0:00:00


In [2]:
!pip install langchain-community
!ollama pull llama3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
/bin/bash: line 1: ollama: command not found


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "tool-using-agent"
os.environ["LANGCHAIN_TRACING_V2"] = "true"


Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


## STEP 2: Define the tool

In [6]:
@tool
def calculator(expression: str) -> str:
    """Evaluates math expressions like 45 * (12 + 8)"""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"


## STEP 3: Define state

In [7]:
class GraphState(TypedDict):
    input: str
    result: str
    final_answer: str

## STEP 4: Define nodes

In [8]:
def call_calculator(state: GraphState) -> GraphState:
    result = calculator.invoke(state["input"])
    return {"result": result}

def format_output(state: GraphState) -> GraphState:
    return {"final_answer": f"The answer is: {state['result']}"}


## STEP 5: Build LangGraph with state schema

In [9]:
builder = StateGraph(GraphState)

builder.add_node("calc_node", RunnableLambda(call_calculator))
builder.add_node("format_node", RunnableLambda(format_output))

builder.set_entry_point("calc_node")
builder.add_edge("calc_node", "format_node")
builder.set_finish_point("format_node")

graph = builder.compile()


## STEP 6: Run graph

In [14]:
result = graph.invoke({"input": "60 * (12 + 8)"})
print(" Final Answer:", result["final_answer"])

 Final Answer: The answer is: 1200
